<a href="https://colab.research.google.com/github/tayfununal/Normalizing-Flows/blob/main/ode_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp

plt.style.use('seaborn-poster')

%matplotlib inline

def F(t,s):       # Diff eq F(t,s) = 0
  return [np.cos(t), -np.sin(t)]

t_eval = np.arange(0, 2*np.pi, 0.01)
sol = solve_ivp(F, [0, 2*np.pi], [0., 1.], t_eval=t_eval, dense_output=True)


plt.figure(figsize = (12, 6))

plt.subplot(221)
plt.plot(sol.t, sol.y[0])
plt.xlabel('t')
plt.ylabel('H[0] = sin(t)')

plt.subplot(222)
plt.plot(sol.t, sol.y[0] - np.sin(sol.t))
plt.xlabel('t')
plt.ylabel('H[0] - sin(t)')

plt.subplot(223)
plt.plot(sol.t, sol.y[1])
plt.xlabel('t')
plt.ylabel('H[1] = cos(t)')

plt.subplot(224)
plt.plot(sol.t, sol.y[1] - np.cos(sol.t))
plt.xlabel('t')
plt.ylabel('H[1] - cos(t)')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (12, 8))
plt.plot(sol.y.T[:, 0], sol.y.T[:, 1])
plt.xlabel('q')
plt.ylabel('p')
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense

In [ ]:
y = sol.y.T # etiketlerimiz
y.shape # (q,p)

In [ ]:
t = sol.t
t.shape

In [ ]:
t = t.reshape(([t.shape[0],1]))
shuffle = np.concatenate((y,t), axis=1)
np.random.shuffle(shuffle)
shuffle

In [ ]:
y = shuffle[:,:2]
t = shuffle[:,2]
print('y:',y, '\nt:',t)

In [ ]:
from tensorflow import keras
import tensorflow as tf

class ChildDense(keras.layers.Layer):
  
  def __init__(self, units, activation=None):
    super().__init__()
    self.units = units
    self.activation = activation
    
  def call(self, inputs):
    y = tf.constant([[0,1]]) + f(t) * inputs
    if self.activation is not None:
      y = self.activation(y)
    return y

  def f(t):
    return 1 - tf.math.exp(-t**2)

In [ ]:
"""tf.random.set_seed(39)
np.random.seed(39)"""
girdi = Input(shape=(1,))

hidden = Dense(512, activation= 'tanh')(girdi)
hidden = Dense(512, activation= 'tanh')(hidden)
hidden = Dense(2)(hidden)


model = Model(inputs=girdi, outputs=hidden)

In [ ]:
from tensorflow.python.framework import dtypes
import keras.backend as K

def custom_loss_wrapper(girdi):
  def custom_loss(y_true, y_pred):
      j = tf.shape(y_pred)
      print(j)
      #print(K.print_tensor(girdi[:j]))

      #(tf.constant([[1.]]) - tf.math.multiply(tf.reshape(tf.cast(tf.math.exp(-girdi),dtype=tf.float32),(629,1)), y_pred))
      return tf.reduce_sum((y_true - y_pred)**2)
  return custom_loss

In [ ]:
model.summary()

In [ ]:
"""import keras.backend as K
def custom_loss(y_true, y_pred):

    #y_true = K.print_tensor(y_true, message='y_true = ')
    #y_pred = K.print_tensor(y_pred, message='y_pred = ')

    y_true = tf.reduce_sum((y_true**2)/2)
    y_pred = tf.reduce_sum((y_pred**2)/2)

    loss = (y_true - y_pred)**2

    return loss
"""

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer='adam', loss=custom_loss_wrapper(t))

In [ ]:
model.fit(x=t, y= y, epochs=1, batch_size=32)

In [ ]:
t_deneme = np.arange(0, 2*np.pi, 0.02).reshape((315,1))
pred = model.predict(t_deneme)

In [ ]:
plt.figure(figsize = (12, 8))
plt.plot(pred[:, 0], pred[:, 1])
plt.plot(sol.y.T[:, 0], sol.y.T[:, 1])
plt.xlabel('q')
plt.ylabel('p')
plt.show()

In [ ]:
a = np.array([[1,2,3,4,5]]).T* np.ones((5,2))
a

In [ ]:
a [2:4]